In [1]:
# defining functions
import json 
import math

def getmeasurementTimestamp(item):
    return int(item['measurementTimestamp'])

def getProcessingTimestamp(item):
    return int(item['processingTimestamp'])

def get_x_error(item): #the error in the data is the stdev of the sample, we compute the error of the estimation (the sample mean)
    return item['value']['averagecoordinate']['error']['coordinates'][0]/math.sqrt(item['value']['trackeeHistory']['nMeasurements'])

def get_y_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][1]/math.sqrt(item['value']['trackeeHistory']['nMeasurements'])

def get_fitted(item):
     return item['value']['trackeeHistory']['fitStatus']
    
def get_x_sample_error(item): 
    return item['value']['averagecoordinate']['error']['coordinates'][0]
def get_y_sample_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][1]

def get_probChi2(item):
    return item['value']['trackeeHistory']['probChi2'] 
def get_Chi2PerDof(item):
    return item['value']['trackeeHistory']['chi2PerDof'] 

def plotHistogramOfDictionary(dictionary, xlabel, ylabel, nbins):
    dictionaryList = []
    for address in dictionary.keys():
        dictionaryList.append(dictionary[address])
    dictArray = np.array(dictionaryList)
    plt.hist(dictArray, bins = nbins)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    axes = plt.gca()
    plt.show()


In [2]:
#reading fitted data from the concert day
for i in [0]:
    data = []
    with open("arena/2015-07-05.json") as f:
    #with open("arena/2015-07-05_raw.json") as f:
        data = f.readlines()

    json_lines = []
    mac_adresses = []
    
    for line in data:
        jsline = json.loads(line)
        json_lines.append(jsline)#now json_lines contains all lines of data
        mac_adresses.append(jsline["value"]["sourceMac"]) # mac_addresses is a list of address per line
        
#sorting by time
json_lines.sort(key = getmeasurementTimestamp) # now json_lines is sorted by time

In [4]:
#computation of error etc (not so important any more)
minTime = getmeasurementTimestamp(json_lines[0])
maxTime = getmeasurementTimestamp(json_lines[len(json_lines) - 1])
print("minTime="+ str(minTime))
print("maxTime="+ str(maxTime))

timeMinutes = (maxTime - minTime)/1000/60

for i in [0]:
    average_x_error = 0
    average_y_error = 0
    count = 0 
    good_lines = []
    number_of_addresses =  len(set(mac_adresses)) #number of mac adresses detected. Not people, because of MAC randomization
    mc_good = []
    delay = 0
    number_of_fitted = 0
    for jsline in json_lines:
        average_x_error += get_x_error(jsline)
        average_y_error += get_y_error(jsline)
        thisdelay = getProcessingTimestamp(jsline) - getmeasurementTimestamp(jsline)
        delay += thisdelay
        if (get_fitted(jsline) == 'FITTED'):
            number_of_fitted +=1
        if get_x_error(jsline) <= 2 and get_y_error(jsline) <= 2:             
            good_lines.append(jsline)
            mc_good.append(jsline["value"]["sourceMac"])

    number_of_lines = len(json_lines)
    number_of_addresses_good = len(set(mc_good)) # number of addresses where error < 2m 
    average_x_error /= number_of_lines
    average_y_error /= number_of_lines
    delay /= number_of_lines
    
    print ('time in minutes:' + str(timeMinutes)) 
    print ('number of lines: ' + str(number_of_lines))
    print ('average x error: ' + str(average_x_error))
    print ('average y error: ' + str(average_y_error))    
    print('number of addresses the whole day:' + str(number_of_addresses))     
    #print('number of estimations with stdev < 2m:' + str(number_of_addresses_good))    #this is not correct 
    print('average delay in ms:' + str(delay))
    print('number of fitted:' + str(number_of_fitted)) 

minTime=1436047367297
maxTime=1436133804311
time in minutes:1440.6169
number of lines: 772867
average x error: 3342.886813949892
average y error: 3697.2146526257307
number of addresses the whole day:30344
average delay in ms:10034.32547644032
number of fitted:772867


In [5]:
#computing FirstTimeSeen and LastTimeSeen for every address
concertFinishedTimestamp=minTime+1000*60*60*8#timestamp until 8 am
FirstTimeSeen=dict()
LastTimeSeen=dict()

for jsline in json_lines:
    address = jsline["value"]["sourceMac"]
    time =  getmeasurementTimestamp(jsline)
    if address in FirstTimeSeen:        
        if time < FirstTimeSeen[address]:
            FirstTimeSeen[address] = time
    else:
        FirstTimeSeen[address] = time
        
    if address in LastTimeSeen:         
        if time > LastTimeSeen[address]:
            LastTimeSeen[address] = time
    else:
        LastTimeSeen[address] = time 

In [6]:
#Computing dwell time, number of persistent addresses, and number of addresses visible in every second
DwellTime = dict()
DwellTimeDuringConcert = dict()
numberOfAdressesAtConcert=0
for address in LastTimeSeen.keys():    
    DwellTime[address] = int((LastTimeSeen[address] - FirstTimeSeen[address]) /1000/60) # in minutes
    if LastTimeSeen[address] < concertFinishedTimestamp:
        numberOfAdressesAtConcert += 1
        DwellTimeDuringConcert[address] = DwellTime[address]  
print('number of addresses detected during concert hours:')
print(numberOfAdressesAtConcert)
longTermAddresses=[]
numberOfAddresses = []
AddressesInSec = []
for jsline in json_lines:
    sec = int(math.floor((getmeasurementTimestamp(jsline)- minTime)/1000))
    #print(str(sec))
    address = jsline["value"]["sourceMac"]
    if DwellTime[address] > 10:#  i.e. >10 minutes 
        longTermAddresses.append(address)
        if len(AddressesInSec) <= sec:
            while len(AddressesInSec) <= sec:
                AddressesInSec.append([])                
            AddressesInSec[sec].append(address)                       
        else:
            if address not in AddressesInSec[sec]:                                
                AddressesInSec[sec].append(address)
for setje in AddressesInSec:
    numberOfAddresses.append(len(setje)) 
longTermCount = len(set(longTermAddresses))                
print("Long term persistent addresses (>10 min)= " + str(longTermCount))
average = 0            
maxN = 0
for addresses in AddressesInSec:            
    average += len(addresses) 
    maxN = max(len(addresses), maxN)
average /= len(AddressesInSec)   
print("averageNumberOfAddressesPerSecondAtConcert " + str(average))
print(maxN)


number of addresses detected during concert hours:
28847
Long term persistent addresses (>10 min)= 10589
averageNumberOfAddressesPerSecondAtConcert 4.615643582683542
36


In [37]:
plotHistogramOfDictionary(DwellTimeDuringConcert, "dwell time(minutes)", "number of addresses", 50)

In [23]:
#drawing how many addresses per second are visible (later there is another way to compute this)
import matplotlib.pyplot as plt
plt.plot(numberOfAddresses)
plt.ylabel('addresses present')
plt.xlabel('sec')
axes = plt.gca()
axes.set_xlim([0,33000])
axes.set_ylim([0,max(numberOfAddresses)])
plt.show()

In [31]:
#unnecessary computations, same as histogram
n_addresses_staying_this_much = []
maxDwellTime = max(DwellTimeDuringConcert.values())
print("maxDwellTime during the whole day=" + str(maxDwellTime) + " minutes")
for i in range(maxDwellTime + 1):   
    n_addresses_staying_this_much.append(0)
    
for address in DwellTimeDuringConcert.keys():  
        n_addresses_staying_this_much[DwellTime[address]] +=1
#for i in range(0,470):
   # print(str(n_addresses_staying_this_much[i]))    
n_addresses_staying_less_than_10min =  sum(n_addresses_staying_this_much[:10])

n_addresses_staying_more_than_10min = numberOfAdressesAtConcert - n_addresses_staying_less_than_10min


print("number of addresses staying less than 10 min during concert hours  = " + str(n_addresses_staying_less_than_10min))
print("number of addresses staying more than 10 min during concert hours  = " + str(n_addresses_staying_more_than_10min))
 

maxDwellTime during the whole day=470 minutes
number of addresses staying less than 10 min during concert hours  = 18438
number of addresses staying more than 10 min during concert hours  = 10409


In [34]:
#plotting dwell time distribution, unnecessary
import matplotlib.pyplot as plt
plt.plot(n_addresses_staying_this_much)
plt.ylabel('number of addresses 05 july')
plt.xlabel('dwell time in minutes')
axes = plt.gca()
#axes.set_xlim([0,470])
#axes.set_ylim([0,200])
#axes.set_ylim([0,max(n_addresses_staying_this_much)])
plt.show()

In [ ]:
106/8

In [7]:
Randomized = dict()
PersistentRandomized = dict()
count0 = 0
count1 = 0
for line in json_lines:
    address = line["value"]["sourceMac"]
    if line["value"]["trackeeHistory"]["localMac"] == 1 :
        count1 +=1
        Randomized[address] = 1
        if DwellTime[address] > 10: 
            if LastTimeSeen[address] < concertFinishedTimestamp:
                PersistentRandomized[address] = 1 
    else:
        count0 +=1
        Randomized[address] = 0
        if DwellTime[address] > 10: 
            if LastTimeSeen[address] < concertFinishedTimestamp:
                PersistentRandomized[address] = 0 

zeros=0
ones=0
zerosPersistent = 0
onesPersistent = 0
for key in Randomized.keys():
    if Randomized[key]==0:
        zeros +=1
    else:
        ones +=1
for key in PersistentRandomized.keys():
    if PersistentRandomized[key]==0:
        zerosPersistent +=1
    else:
        onesPersistent +=1        
print("total number of lines with localMac == 1: " + str(count1))
print("total number of lines with localMac == 0: " + str(count0))
print("total number of addresses with localMac == 1: " + str(ones))
print("total number of addresses with localMac == 0: " + str(zeros))
print("total number of persistent addresses with localMac == 1: " + str(onesPersistent))
print("total number of persistent addresses with localMac == 0: " + str(zerosPersistent))
      


total number of lines with localMac == 1: 60024
total number of lines with localMac == 0: 712843
total number of addresses with localMac == 1: 15554
total number of addresses with localMac == 0: 14790
total number of persistent addresses with localMac == 1: 483
total number of persistent addresses with localMac == 0: 9769


In [8]:
DwellTimeConcertRandomized = dict()
DwellTimeConcertNonRandomized = dict()
for key in DwellTimeDuringConcert.keys():
    if Randomized[key] == 0:
        DwellTimeConcertNonRandomized[key] = DwellTimeDuringConcert[key]
    else:
        DwellTimeConcertRandomized[key] = DwellTimeDuringConcert[key]

    

In [47]:
plotHistogramOfDictionary(DwellTimeConcertNonRandomized, "dwell time(minutes) of non-randomized addresses", "number of addresses", 50)

In [49]:
plotHistogramOfDictionary(DwellTimeConcertRandomized, "dwell time(minutes) of randomized addresses", "number of addresses",200)

In [9]:
#double-check:another approach to computing visible addresses per second
Visible = dict()
for line in json_lines:
    address = line["value"]["sourceMac"]
    timeSec = int(math.floor((getmeasurementTimestamp(line)- minTime)/1000))
    if timeSec in Visible.keys():        
        Visible[timeSec].append(address)
    else:
        Visible[timeSec] = []
        Visible[timeSec].append(address)

maxSec = max(Visible.keys())    
print(maxSec)
for sec in range(0,maxSec):
    if sec not in Visible.keys():
        Visible[sec] = []
NVisible = []
for sec in range(0,86437):
    NVisible.append(len(set(Visible[sec])))
    

86437


In [10]:
def ComputeVisiblePerSec(json_lines, localMac):
    Visible = dict()
    for line in json_lines:
        address = line["value"]["sourceMac"]
        timeSec = int(math.floor((getmeasurementTimestamp(line)- minTime)/1000))
        if timeSec in Visible.keys():  
            if Randomized[address] == localMac:
                Visible[timeSec].append(address)
        else:
            Visible[timeSec] = []
            if Randomized[address] == localMac:
                Visible[timeSec].append(address)
    maxSec = max(Visible.keys())        
    for sec in range(0,maxSec):
        if sec not in Visible.keys():
            Visible[sec] = []
    NVisible = []
    for sec in range(0,maxSec):
        NVisible.append(len(set(Visible[sec])))
    return NVisible

In [11]:
VisiblePerSecondNonRandomized = ComputeVisiblePerSec(json_lines, 0)
VisiblePerSecondRandomized = ComputeVisiblePerSec(json_lines, 1) 

In [69]:
#drawing how many addresses per second are visible
import matplotlib.pyplot as plt
plt.plot(NVisible)
plt.ylabel('visible addresses')
plt.xlabel('time(sec)')
axes = plt.gca()
axes.set_xlim([0,30000])
axes.set_ylim([0,max(NVisible)])
plt.show()

In [70]:
#drawing how many nonrandomized addresses per second are visible
plt.plot(VisiblePerSecondNonRandomized)
plt.ylabel('visible non-randomized addresses')
plt.xlabel('time(sec)')
axes = plt.gca()
axes.set_xlim([0,30000])
axes.set_ylim([0,max(VisiblePerSecondNonRandomized)])
plt.show()

In [71]:
#drawing how many randomized addresses per second are visible
plt.plot(VisiblePerSecondRandomized)
plt.ylabel('visible randomized addresses')
plt.xlabel('time(sec)')
axes = plt.gca()
axes.set_xlim([0,30000])
axes.set_ylim([0,max(VisiblePerSecondRandomized)])
plt.show()

In [11]:
# checking the trajectory of a certain address, error used is sample error (not error of the estimation of the mean) 
lines = []#will contain only the lines for that address
for line in json_lines:
    address = line["value"]["sourceMac"]
    #if (address == '27e573c8-1640-4ea8-86d8-0733c800e9cd'):#this is the address we are checking for, non-randomized
    if (address == '3620f07e-a673-4648-bd9b-a0681d574d6f'):
    #if (address == '8b8a2356-d11e-4bd5-bb35-d8370bf48b1e'):#randomized address    
        lines.append(line)
       
x_coord = []
y_coord = []
times = []
x_errors = []
y_errors = []
for line in lines:  
    alpha = get_probChi2(line)    
    if ( alpha > 0.4) and (get_x_sample_error(line) <10 ) and  get_Chi2PerDof(line) <1.1 :    
        coordinates = line["value"]["averagecoordinate"]["avg"]["coordinates"]
        time = math.floor(line["measurementTimestamp"]/1000)      
        if time not in times:       
            x_coord.append( coordinates[0])
            y_coord.append(coordinates[1])
            x_errors.append(get_x_sample_error(line)*2)# for a 95% confidence
            y_errors.append(get_y_sample_error(line)*2)# for a 95% confidence
            times.append(time)
    
print(len(x_errors)) 
print(len(x_coord))

84
84


In [12]:
# checking the trajectory of a certain address, error used is error of the estimation of the mean
lines = []#will contain only the lines for that address
for line in json_lines:
    address = line["value"]["sourceMac"]
    #if (address == '27e573c8-1640-4ea8-86d8-0733c800e9cd'):#this is the address we are checking for, nonrandomized
    #if address =='e1deda99-163f-4b68-9ade-e1f05d070bf2' : # the largest one, probably from the organizers  
    #if (address == '3620f07e-a673-4648-bd9b-a0681d574d6f'):
    if (address == '8b8a2356-d11e-4bd5-bb35-d8370bf48b1e'):#randomized address  
        lines.append(line)
       
x_coord = []
y_coord = []
times = []
x_errors = []
y_errors = []
for line in lines:
    alpha = get_probChi2(line)
    if ( alpha > 0.3) and (get_x_error(line) <5 ) and  get_Chi2PerDof(line) <1.2 :    
        coordinates = line["value"]["averagecoordinate"]["avg"]["coordinates"]
        time = math.floor(line["measurementTimestamp"]/1000)        
        if time not in times:
            x_coord.append( coordinates[0])
            y_coord.append(coordinates[1])
            x_errors.append(get_x_error(line)*2)# for a 95% confidence
            y_errors.append(get_y_error(line)*2)# for a 95% confidence
            times.append(time)
    
print(len(x_errors)) 
print(len(x_coord))

477
477


In [16]:
import numpy as np
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(times, x_coord, yerr=x_errors, fmt='o')
ax0.set_title('x-coordinate with error')
plt.xlabel('time(sec)')

ax1.errorbar(times, y_coord, yerr=y_errors, fmt='o')
ax1.set_title('y-coordinate with error')

plt.show()


In [4]:
coor = dict()
count =0
for x in x_coord:
    key = times[count]
    coor[key] = x
    count +=1
plotHistogramOfDictionary(coor,  'x_coord', 'freq', 200)    
    

NameError: name 'x_coord' is not defined